# Wald solution

## Basic initialization of the lorentzian manifold

Initialization of the Kerr manifold $\mathcal M_{Kerr}$ atlas with Boyer-Lindquist chart (BL) and of the metric tensor $g$ :

In [1]:
%display latex

M = Manifold(4, 'M', latex_name=r'\mathcal{M}', structure='Lorentzian')
BL.<t,r,th,ph> = M.chart(r"t r th:(0,pi):\theta ph:(0,2*pi):\phi") #boyer-lindquist chart definition

var('m, a', domain='real') #definition of Kerr metric parameters

g = M.metric() #we put the label g on the metric tensor

rho2 = r^2 + (a*cos(th))^2
Delta = r^2 -2*m*r + a^2

g[0,0] = -(1-2*m*r/rho2)
g[0,3] = -2*a*m*r*sin(th)^2/rho2
g[1,1], g[2,2] = rho2/Delta, rho2
g[3,3] = (r^2+a^2+2*m*r*(a*sin(th))^2/rho2)*sin(th)^2

g.display()

g = (2*m*r/(a^2*cos(th)^2 + r^2) - 1) dt*dt - 2*a*m*r*sin(th)^2/(a^2*cos(th)^2 + r^2) dt*dph + (a^2*cos(th)^2 + r^2)/(a^2 - 2*m*r + r^2) dr*dr + (a^2*cos(th)^2 + r^2) dth*dth - 2*a*m*r*sin(th)^2/(a^2*cos(th)^2 + r^2) dph*dt + (2*a^2*m*r*sin(th)^2/(a^2*cos(th)^2 + r^2) + a^2 + r^2)*sin(th)^2 dph*dph

Definition of the Levi Civita connection $\nabla$:

In [2]:
nabla = g.connection()

## Initialization of the orthonormal tetrad

Definition of the orthonormal tetrad frame $e_{(a)}$ (Chandrasekar notation) and its dual frame $e^{(a)} = \eta^{(a)(b)}g(e_{(b)},-)$ ($\omega^{a}$ in Wald's paper):

In [3]:
#definition of the coframe tetrad fields (as listed in the article)

de0 = M.one_form(sqrt(Delta/rho2), 0, 0, -a*(sin(th)^2)*sqrt(Delta/rho2), frame=BL.frame(), name='e^{(0)}')
de1 = M.one_form(0, sqrt(rho2/Delta), 0, 0, frame=BL.frame(), name='e^{(1)}')
de2 = M.one_form(0, 0, sqrt(rho2), 0, frame=BL.frame(), name='e^{(2)}')
de3 = M.one_form(-(sin(th)/sqrt(rho2))*a, 0, 0, (sin(th)/sqrt(rho2))*(r^2+a^2), frame=BL.frame(), name='e^{(3)}')

#obtaining the tetrad frame vector fields using musical isomorphism of the metric

e0 = -de0.up(g) #the zero component must be multiplied by a minus because of the lowering of the tetrad index with eta_{(a)(b)}
e1 = de1.up(g)
e2 = de2.up(g)
e3 = de3.up(g)


e = M.vector_frame('e', (e0, e1, e2, e3)) #costruction of the tetrad collection object e_{a}
de = e.coframe() #construction of the tetrad coframe object e^{a}      

Orthonormality check $g(e_{(a)},e_{(b)}) = \eta_{(a)(b)}$ :

In [4]:
from sage.tensor.modules.comp import Components

scalar_field_algebra = M.scalar_field_algebra()

#definition of eta_{(a)(b)}
eta = Components(scalar_field_algebra, e, 2)
eta[0,0] = M.scalar_field({BL: -1}, name='-1')
eta[1,1] = M.scalar_field({BL: 1}, name='1')
eta[2,2] = M.scalar_field({BL: 1}, name='1')
eta[3,3] = M.scalar_field({BL: 1}, name='1')

for i in range(0,4):
    for j in range(0,4):
        if (g(e[i],e[j]) != eta[i,j]):
            print("error")

Check that the one forms given are equal to the one computed by Sage:

In [5]:
if (de[0] != de0 or de[1] != de1 or de[2] != de2 or de[3] != de3):
    print("error")

## Wald field solution definition

Definition of the Wald field solution $A = \frac{B_{0}}{2}(\psi + 2a\eta)$, where $\eta$ and $\psi$ are the 1-forms naturally associated with musical isomorphism respectively to the two Killing vector fields $\frac{\partial}{\partial t}$ and $\frac{\partial}{\partial \phi}$ :

In [6]:
var('B_0')
A = M.vector_field(name = "A")
A = (B_0/2)*(BL.frame()[3].down(g)+(2*a)*BL.frame()[0].down(g)) #BL.frame() gives the coordinate base of the chart before the dot, in this case Boyer-Lindquist chart
A.display()

(B_0*a*m*r - B_0*a*r^2 - (B_0*a^3 - B_0*a*m*r)*cos(th)^2)/(a^2*cos(th)^2 + r^2) dt + 1/2*(2*B_0*a^2*m*r*sin(th)^4 - (4*B_0*a^2*m*r - B_0*a^2*r^2 - B_0*r^4 - (B_0*a^4 + B_0*a^2*r^2)*cos(th)^2)*sin(th)^2)/(a^2*cos(th)^2 + r^2) dph

## Check of Comar integrals computations

Calculation of $\star d\eta$ :

In [7]:
star_d_eta =  BL.frame()[0].down(g).exterior_derivative().hodge_dual()

Component $(\star d\eta)_{\theta \phi}$ :

In [8]:
star_d_eta[2,3].expr().maxima_methods().rootscontract().full_simplify() #correct (see my notes)

2*(a^4*m - m*r^4 - (a^4*m + a^2*m*r^2)*sin(th)^2)*abs(sin(th))/(a^4*cos(th)^4 + 2*a^2*r^2*cos(th)^2 + r^4)

$\lim_{r \to \infty} \ (\star d\eta)_{\theta \phi}$ :

In [9]:
star_d_eta[2,3].expr().limit(r = infinity).maxima_methods().rootscontract().full_simplify() #correct (see my notes)

-2*m*abs(sin(th))

Calculation of $\star d\psi$ :

In [10]:
star_d_psi = BL.frame()[3].down(g).exterior_derivative().hodge_dual()

Component $(\star d\psi)_{\theta \phi}$ :

In [11]:
star_d_psi[2,3].expr().maxima_methods().rootscontract().full_simplify() #correct (see my notes)

2*(a^3*m*r^2 + 3*a*m*r^4 - (a^5*m - a^3*m*r^2)*cos(th)^2)*sqrt(-cos(th)^2 + 1)*sin(th)^2/(a^4*cos(th)^4 + 2*a^2*r^2*cos(th)^2 + r^4)

$\lim_{r \to \infty} \ (\star d\psi)_{\theta \phi}$ :

In [12]:
star_d_psi[2,3].expr().limit(r = infinity).maxima_methods().rootscontract().full_simplify() #correct (see my notes)

6*a*m*abs(sin(th))*sin(th)^2

## Faraday's tensor's behaviour

Calculation of the Faraday tensor $F=dA$ (exterior derivative of $A$) :

In [13]:
F = A.exterior_derivative()
F[:]

[                                                                                                                                                                                                                     0                                                                                                 -(B_0*a^3*m*cos(th)^4 - B_0*a*m*r^2 + (B_0*a^3*m - B_0*a*m*r^2)*cos(th)^2)/(a^4*cos(th)^4 + 2*a^2*r^2*cos(th)^2 + r^4)                                                                                                                             -2*(B_0*a^3*m*r - B_0*a*m*r^3)*cos(th)*sin(th)/(a^4*cos(th)^4 + 2*a^2*r^2*cos(th)^2 + r^4)                                                                                                                                                                                                                      0]
[                                                                                                 (B_0*a^3*m*cos(th)^4 - B_0*a*m*r^2 + (B_0*a^3*m - B_0*a*m*r^2)*cos(th)^2)/(a^4*cos(th)^4 + 2*a^2*r^2*cos(th)^2 + r^4)                                                                                                                                                                                                                      0                                                                                                                                                                                                                      0                    ((B_0*a^4*m*cos(th)^2 - B_0*a^2*m*r^2)*sin(th)^4 + (B_0*a^4*r*cos(th)^4 + 2*B_0*a^2*m*r^2 + B_0*r^5 - 2*(B_0*a^4*m - B_0*a^2*r^3)*cos(th)^2)*sin(th)^2)/(a^4*cos(th)^4 + 2*a^2*r^2*cos(th)^2 + r^4)]
[                                                                                                                             2*(B_0*a^3*m*r - B_0*a*m*r^3)*cos(th)*sin(th)/(a^4*cos(th)^4 + 2*a^2*r^2*cos(th)^2 + r^4)                                                                                                                                                                                                                      0                                                                                                                                                                                                                      0  ((B_0*a^6 - 2*B_0*a^4*m*r + B_0*a^4*r^2)*cos(th)^5 + 2*(B_0*a^4*r^2 - 2*B_0*a^2*m*r^3 + B_0*a^2*r^4)*cos(th)^3 - (2*B_0*a^4*m*r - B_0*a^2*r^4 - B_0*r^6)*cos(th))*sin(th)/(a^4*cos(th)^4 + 2*a^2*r^2*cos(th)^2 + r^4)]
[                                                                                                                                                                                                                     0                   -((B_0*a^4*m*cos(th)^2 - B_0*a^2*m*r^2)*sin(th)^4 + (B_0*a^4*r*cos(th)^4 + 2*B_0*a^2*m*r^2 + B_0*r^5 - 2*(B_0*a^4*m - B_0*a^2*r^3)*cos(th)^2)*sin(th)^2)/(a^4*cos(th)^4 + 2*a^2*r^2*cos(th)^2 + r^4) -((B_0*a^6 - 2*B_0*a^4*m*r + B_0*a^4*r^2)*cos(th)^5 + 2*(B_0*a^4*r^2 - 2*B_0*a^2*m*r^3 + B_0*a^2*r^4)*cos(th)^3 - (2*B_0*a^4*m*r - B_0*a^2*r^4 - B_0*r^6)*cos(th))*sin(th)/(a^4*cos(th)^4 + 2*a^2*r^2*cos(th)^2 + r^4)                                                                                                                                                                                                                      0]

Computing the fully contravariant version of $F$ using musical isomorphism twice:

In [14]:
F_uu = F.up(g,0).up(g,1)
F_uu[:]

[                                                                                                                                                                                                                                0                                -(B_0*a^3*m*r^2 - B_0*a*m*r^4 - (B_0*a^5*m - B_0*a^3*m*r^2)*cos(th)^4 - (B_0*a^5*m + 2*B_0*a^3*m*r^2 - 3*B_0*a*m*r^4)*cos(th)^2)/(a^6*cos(th)^6 + 3*a^4*r^2*cos(th)^4 + 3*a^2*r^4*cos(th)^2 + r^6)                                                                                                         2*(B_0*a^3*m*r*cos(th)^3 + B_0*a^3*m*r*cos(th))*sin(th)/(a^6*cos(th)^6 + 3*a^4*r^2*cos(th)^4 + 3*a^2*r^4*cos(th)^2 + r^6)                                                                                                                                                                                                                                 0]
[                                (B_0*a^3*m*r^2 - B_0*a*m*r^4 - (B_0*a^5*m - B_0*a^3*m*r^2)*cos(th)^4 - (B_0*a^5*m + 2*B_0*a^3*m*r^2 - 3*B_0*a*m*r^4)*cos(th)^2)/(a^6*cos(th)^6 + 3*a^4*r^2*cos(th)^4 + 3*a^2*r^4*cos(th)^2 + r^6)                                                                                                                                                                                                                                 0                                                                                                                                                                                                                                 0                             (B_0*a^2*m*r^2 - 2*B_0*m*r^4 + B_0*r^5 - (B_0*a^4*m - B_0*a^4*r)*cos(th)^4 - (B_0*a^4*m + B_0*a^2*m*r^2 - 2*B_0*a^2*r^3)*cos(th)^2)/(a^6*cos(th)^6 + 3*a^4*r^2*cos(th)^4 + 3*a^2*r^4*cos(th)^2 + r^6)]
[                                                                                                       -2*(B_0*a^3*m*r*cos(th)^3 + B_0*a^3*m*r*cos(th))*sin(th)/(a^6*cos(th)^6 + 3*a^4*r^2*cos(th)^4 + 3*a^2*r^4*cos(th)^2 + r^6)                                                                                                                                                                                                                                 0                                                                                                                                                                                                                                 0 (B_0*a^4*cos(th)*sin(th)^4 - 2*(B_0*a^4 - B_0*a^2*m*r + B_0*a^2*r^2)*cos(th)*sin(th)^2 + (B_0*a^4 - 4*B_0*a^2*m*r + 2*B_0*a^2*r^2 + B_0*r^4)*cos(th))/((a^6*cos(th)^6 + 3*a^4*r^2*cos(th)^4 + 3*a^2*r^4*cos(th)^2 + r^6)*sin(th))]
[                                                                                                                                                                                                                                0                            -(B_0*a^2*m*r^2 - 2*B_0*m*r^4 + B_0*r^5 - (B_0*a^4*m - B_0*a^4*r)*cos(th)^4 - (B_0*a^4*m + B_0*a^2*m*r^2 - 2*B_0*a^2*r^3)*cos(th)^2)/(a^6*cos(th)^6 + 3*a^4*r^2*cos(th)^4 + 3*a^2*r^4*cos(th)^2 + r^6)                                                    -(B_0*a^4*cos(th)^5 - 2*(B_0*a^2*m*r - B_0*a^2*r^2)*cos(th)^3 - (2*B_0*a^2*m*r - B_0*r^4)*cos(th))/((a^6*cos(th)^6 + 3*a^4*r^2*cos(th)^4 + 3*a^2*r^4*cos(th)^2 + r^6)*sin(th))                                                                                                                                                                                                                                 0]

Check that this $F$ is a solution of $\nabla_{\mu}F^{\mu\nu} = 0$ :

In [15]:
nabla(F_uu)["^ab_a"] == 0 #correct

True

Check of the behaviour of the non trivial components of $F$ in the limit for $r >> 1$ :

$F_{tr} \to 0$ for $r \to \infty$ :

In [16]:
F[0,1].expr().limit(r=infinity) #correct

0

$F_{t\theta} \to 0$ for $r \to \infty$ :

In [17]:
F[0,2].expr().limit(r=infinity) #correct

0

$F_{r\phi} \text{~} B_{0}r\sin^{2}\theta$ for $r >> 1$ :

In [18]:
F_13(r) = F[1,3].expr().function(r)
var("u")
F_13(u) = F_13(1/u)
F_13 = F_13.series(u,2).full_simplify() #taylor series expansion for small u, u = 1/r
F_13(r) = F_13(1/r)
F_13 #correct

r |--> (B_0*sin(th)^2 + Order(r^(-2))/r)*r

$F_{\theta\phi} \text{~} B_{0}r^{2}\sin\theta\cos\theta$ for $r >> 1$ :

In [19]:
F_23(r) = F[2,3].expr().function(r)
F_23(u) = F_23(1/u)
F_23 = F_23.series(u,1).full_simplify() #taylor series expansion for small u, u = 1/r
F_23(r)= F_23(1/r)
F_23 #correct

r |--> (B_0*cos(th)*sin(th) + B_0*a^2*cos(th)*sin(th)/r^2 + Order(1/r)/r^2)*r^2

Check of the behaviour of the non trivial components of $F$ fully contravariant and $\vec E$ in the limit for $r >> 1$ :

$F^{tr} = E^{r} \to 0$ for $r \to \infty$ :

In [20]:
F_uu[0,1].expr().limit(r=infinity) #correct

0

$F^{t\theta}=E^{\theta} \to 0$ for $r \to \infty$ :

In [21]:
F_uu[0,2].expr().limit(r=infinity) #correct

0

Therefore we checked that $\vec E \to 0$ for $r \to \infty$ (since $F^{t\phi} = E^{\phi} = 0$ everywhere) as expected.

$F^{r\phi} \text{~} \frac{B_{0}}{r}$ for $r >> 1$ :

In [22]:
F_uu_13(r) = F_uu[1,3].expr().function(r)
F_uu_13(u) = F_uu_13(1/u)
F_uu_13 = F_uu_13.series(u,2).full_simplify() #taylor series expansion for small u, u = 1/r
F_uu_13(r) = F_uu_13(1/r)
F_uu_13 #correct

r |--> B_0/r + Order(r^(-2))

$F^{\theta\phi} \text{~} \frac{B_{0}\cos\theta}{r^{2}\sin\theta}$ for $r >> 1$ :

In [23]:
F_uu_23(r) = F_uu[2,3].expr().function(r)
F_uu_23(u) = F_uu_23(1/u)
F_uu_23 = F_uu_23.series(u,3).full_simplify() #taylor series expansion for small u, u = 1/r
F_uu_23(r) = F_uu_23(1/r)
F_uu_23 #correct

r |--> (Order(r^(-3))*sin(th) + B_0*cos(th)/r^2)/sin(th)

Display of the components of $F$ with respect to the tetrad frame $e_{(a)}$ :

In [24]:
F.display(e) #correct, they are equal to Wald's paper

-(2*B_0*a^3*m - 2*B_0*a*m*r^2 + (B_0*a^3*m - B_0*a^3*r)*sin(th)^4 - (3*B_0*a^3*m - B_0*a^3*r - B_0*a*m*r^2 - B_0*a*r^3)*sin(th)^2)/(a^4*cos(th)^4 + 2*a^2*r^2*cos(th)^2 + r^4) e^0/\e^1 - sqrt(a^2 - 2*m*r + r^2)*B_0*a*cos(th)*sin(th)/(a^2*cos(th)^2 + r^2) e^0/\e^2 + sqrt(a^2 - 2*m*r + r^2)*B_0*r*sin(th)/(a^2*cos(th)^2 + r^2) e^1/\e^3 - ((B_0*a^4 - 2*B_0*a^2*m*r + B_0*a^2*r^2)*cos(th)*sin(th)^2 - (B_0*a^4 - 4*B_0*a^2*m*r + 2*B_0*a^2*r^2 + B_0*r^4)*cos(th))/(a^4*cos(th)^4 + 2*a^2*r^2*cos(th)^2 + r^4) e^2/\e^3

Calculation of the scalar $F_{\mu\nu}F^{\mu\nu}$ :

In [25]:
FF_scalar = F.contract(0,1,F_uu,0,1)
FF_scalar.display()

M --> R
(t, r, th, ph) |--> -2*(B_0^2*a^2*m^2*r^4 - 2*B_0^2*a^2*m*r^5 + 2*B_0^2*m*r^7 - B_0^2*r^8 - (B_0^2*a^8 - B_0^2*a^6*m^2)*cos(th)^8 + 2*(B_0^2*a^6*m^2 + B_0^2*a^6*m*r + 3*B_0^2*a^4*m*r^3 - (2*B_0^2*a^6 + 3*B_0^2*a^4*m^2)*r^2)*cos(th)^6 + (B_0^2*a^6*m^2 + 6*B_0^2*a^6*m*r - 12*B_0^2*a^4*m^2*r^2 + 6*B_0^2*a^4*m*r^3 + 4*B_0^2*a^2*m*r^5 - (6*B_0^2*a^4 - B_0^2*a^2*m^2)*r^4)*cos(th)^4 - 2*(3*B_0^2*a^4*m^2*r^2 - 2*B_0^2*a^4*m*r^3 - B_0^2*a^2*m^2*r^4 - 3*B_0^2*a^2*m*r^5 + 2*B_0^2*a^2*r^6 + B_0^2*m*r^7)*cos(th)^2)/(a^8*cos(th)^8 + 4*a^6*r^2*cos(th)^6 + 6*a^4*r^4*cos(th)^4 + 4*a^2*r^6*cos(th)^2 + r^8)

Calculation of $\star F$ :

In [26]:
F_star = F.hodge_dual()

Calculation of the fully contravariant version of $\star F$ using musical isomorphism twice:

In [27]:
F_star_uu = F_star.up(g,0).up(g,1)

Check that $-(\star F)^{tr} = B^{r} \text{~}  B_{0}\cos\theta$ for $r >> 1$ :

In [28]:
-F_star_uu[0,1].expr().limit(r=infinity).maxima_methods().rootscontract().full_simplify() #correct

B_0*abs(sin(th))*cos(th)/sin(th)

Check that $-(\star F)^{t\theta}=B^{\theta}  \text{~} -\frac{B_{0}\sin\theta}{r}$ for $r >> 1$ :

In [29]:
B_th(r) = -F_star_uu[0,2].expr().function(r)
B_th(u) = B_th(1/u)
B_th = B_th.series(u,2).full_simplify() #taylor series expansion for small u, u = 1/r
B_th(r) = B_th(1/r)
B_th #correct

r |--> -B_0*sqrt(cos(th) + 1)*sqrt(-cos(th) + 1)/r + Order(r^(-2))

Check that $-(\star F)^{t\phi} = B^{\phi} = 0$ :

In [30]:
-F_star_uu[0,3].expr() #correct

0

Now if we consider a 3 dimensional Euclidean space and a vector field $\vec B$ with the components obtained above $B^{i}=(B_{0}\cos\theta,-\frac{B_{0}\sin\theta}{r},0)$ in spherical coordinates:

In [42]:
E.<x,y,z> = EuclideanSpace()

cartesian = E.cartesian_coordinates()
spherical.<r,th,ph> = E.spherical_coordinates()

B = B_0*cos(th)*spherical.frame()[1]-(B_0*sin(th)/r)*spherical.frame()[2]
B.display(spherical)

B_0*cos(th) d/dr - B_0*sin(th)/r d/dth

If we check the components of this vector field in cartesian coordinates we get:

In [32]:
B.display(cartesian)

B_0 e_z

Therefore we checked that the magnetic field $\vec B$ for $r >> 1$ is uniform and parallel to the rotation axis (points upward if $B_0 > 0$) of the Kerr solution and has magnitude $|B_{0}|$ as requested.

Calculation of the scalar $(\star F)^{\mu\nu}F_{\mu\nu}$ :

In [33]:
F_star_F_scalar = F_star_uu.contract(0,1,F,0,1)
F_star_F_scalar.display()

M --> R
(t, r, th, ph) |--> -4*((B_0^2*a^7*m - 2*B_0^2*a^5*m^2*r + B_0^2*a^5*m*r^2)*cos(th)^7 + (B_0^2*a^7*m - 4*B_0^2*a^5*m^2*r + 3*B_0^2*a^5*m*r^2 + 2*B_0^2*a^3*m^2*r^3 - 2*B_0^2*a^3*m*r^4)*cos(th)^5 - (2*B_0^2*a^5*m^2*r + 2*B_0^2*a^5*m*r^2 - 4*B_0^2*a^3*m^2*r^3 - 3*B_0^2*a^3*m*r^4 + 3*B_0^2*a*m*r^6)*cos(th)^3 + (2*B_0^2*a^3*m^2*r^3 - 3*B_0^2*a^3*m*r^4 + B_0^2*a*m*r^6)*cos(th))*sqrt(cos(th) + 1)*sqrt(-cos(th) + 1)*sin(th)/(a^8*cos(th)^10 - (a^8 - 4*a^6*r^2)*cos(th)^8 - r^8 - 2*(2*a^6*r^2 - 3*a^4*r^4)*cos(th)^6 - 2*(3*a^4*r^4 - 2*a^2*r^6)*cos(th)^4 - (4*a^2*r^6 - r^8)*cos(th)^2)

Calculation of $(\star F)^{\mu\nu}F_{\mu\nu}$ scalar for $r \to \infty$ (we should expect it to be 0 since in this case $\vec E \to 0$ for $r \to \infty$ and $\frac{1}{4}(\star F)^{\mu\nu}F_{\mu\nu}=\vec E \cdot \vec B$):

In [34]:
F_star_F_scalar.expr().limit(r = infinity) #correct

0

Calculation of $\frac{1}{2}F^{\mu\nu}F_{\mu\nu}$ scalar for $r \to \infty$ (we should expect it to be $B_{0}^{2}$ since in this case $\vec E \to 0$ for $r \to \infty$ and $\frac{1}{2}F^{\mu\nu}F_{\mu\nu}=\vec B^{2} - \vec E^{2}$) :

In [35]:
(1/2)*FF_scalar.expr().limit(r = infinity) #correct

B_0^2

## $\vec E$ and $\vec B$ behaviour for $a = 0$

Check that $\vec E = 0$ everywhere for $a=0$ :

$E^{r}_{a=0}$ :

In [36]:
F_uu[0,1].expr().limit(a=0)

0

$E^{\theta}_{a=0}$ :

In [37]:
F_uu[0,2].expr().limit(a=0)

0

$E^{\phi}_{a=0}$ :

In [38]:
F_uu[0,3].expr().limit(a=0)

0

Now we check the magnetic field $\vec B$

$B^{r}_{a=0}$ :

In [39]:
-F_star_uu[0,1].expr().limit(a=0).maxima_methods().rootscontract().full_simplify()

B_0*abs(sin(th))*cos(th)/sin(th)

$B^{\theta}_{a=0}$ :

In [40]:
-F_star_uu[0,2].expr().limit(a=0).maxima_methods().rootscontract().full_simplify()

-B_0*abs(sin(th))/r

$B^{\phi}_{a=0}$ :

In [41]:
-F_star_uu[0,3].expr().limit(a=0).maxima_methods().rootscontract().full_simplify()

0

Therefore, in the case of no rotation ($a=0$) (Schwarzschild metric) the solution is a uniform magnetic field throughout the whole spacetime with the same features of the asymptotic one in Kerr metric.

So if we turn on rotation ($a\ne0$) and we get closer and closer to the event horizon the effect of rotation and frame dragging becomes stronger and stronger and the asymptotic uniform magnetic field get mixed into a more complicated combination of both eletric and magnetic fields.